<a href="https://colab.research.google.com/github/coderTanisha22/Jarvis_learnings/blob/main/Copy_of_hugging_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers #pretrained AI models access
!pip install torch #pytorch backened use to run models

In [ ]:
!pip install gradio transformers torch #for maing a new web interface for chatbot

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
chatbot_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

In [ ]:
chathis_tokens = None

In [ ]:
def generate_response(messages, chatbot=None):
    global chathis_tokens
    try:
        if isinstance(messages, str):
            messages = [{"role": "user", "content": messages}]

        user_message = messages[-1]["content"]

        user_tokens = tokenizer.encode(user_message + tokenizer.eos_token, return_tensors='pt')
        model_input = torch.cat([chathis_tokens, user_tokens], dim=-1) if chathis_tokens is not None else user_tokens

        response_tokens = chatbot_model.generate(
            model_input,
            max_length=1000,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

        new_tokens = response_tokens[0][model_input.shape[-1]:]
        bot_reply = tokenizer.decode(new_tokens, skip_special_tokens=True)
        messages.append({"role": "assistant", "content": bot_reply})
        chathis_tokens = response_tokens

        return messages

    except Exception as e:
        if not isinstance(messages, list):
            messages = [{"role": "user", "content": str(messages)}]
        messages.append({"role": "assistant", "content": f"[Error] {str(e)}"})
        return messages

In [ ]:
import gradio as gr
gr.ChatInterface(fn=generate_response, type="messages").launch(debug=True) #so that it shows the error messege; it's not hidden